In [1]:
import time

import numpy as np

In [2]:
def load_data(is_test=True):
    if is_test:
        input_txt = 'input_test.txt'
    else:
        input_txt = 'input.txt'
    return np.loadtxt(input_txt, dtype="int", delimiter=",")


# calculate the number of adjacent surfaces.
def calc_adjacent_surfaces(X):
    n_adjacent = 0
    n_cubes = len(X)
    time_start = time.time()
    for i in range(n_cubes):
        for j in range(n_cubes):
            if i != j:
                x1 = X[i]
                x2 = X[j]
                if np.linalg.norm(x1-x2) == 1:
                    n_adjacent += 1
    return n_adjacent


def list_adjacent_surfaces(x):
    return [
        x + [1, 0, 0], x + [-1, 0, 0],
        x + [0, 1, 0], x + [0, -1, 0],
        x + [0, 0, 1], x + [0, 0, -1],
    ]

In [55]:
is_test = False
X = load_data(is_test=is_test)

## part 1.
time_start = time.time()
n_cubes = len(X)
n_adjacent = calc_adjacent_surfaces(X)
n_surfaces = 6*n_cubes - n_adjacent
print(f"after counting up all the sides that aren't connected to another cube")
print(f"the total surface area is {n_surfaces}")
print(f"elapsed time: {time.time()-time_start:.3}[s]")

after counting up all the sides that aren't connected to another cube
the total surface area is 3610
elapsed time: 13.5[s]


In [42]:
# def reduce_options(X, options, adjacent_cube):
#     # check if all the adjacent surfaces are in X.
#     options_ = np.copy(options)
#     for option in options:
#         ids = np.where(np.all(X == option + adjacent_cube, axis=1))
#         if len(ids[0]) == 0:
#             ids_ = np.where(np.all(options_ == option, axis=1))
#             options_ = np.copy(np.delete(options_, ids_[0][0], axis=0))
#     return options_


def np2set(X):
    return set([tuple(x) for x in X])


def search_row(X, x):
    ids = np.where((X == x).all(axis=1))
    return ids[0]


def get_group(x):
    global group
    global options
    for x_ in list_adjacent_surfaces(x):
        ids_X = search_row(X, x_)
        ids_options = search_row(options, x_)
        if len(ids_X) == 0 and len(ids_options) == 1:
            group.append(x_)
            ids = search_row(options, np.array([-1, -1, -1]))
            #print(f"{x_}, {len(ids)}")
            options[ids_options[0]] = np.array([-1, -1, -1])
            get_group(x_)

In [53]:
## part 2. 
is_test = False
X = load_data(is_test=is_test)
#time_start = time.time()

xmin = np.min(X[:, 0])
xmax = np.max(X[:, 0])
ymin = np.min(X[:, 1])
ymax = np.max(X[:, 1])
zmin = np.min(X[:, 2])
zmax = np.max(X[:, 2])
options = []
for x in range(xmin, xmax+1):
    for y in range(ymin, ymax+1):
        for z in range(zmin, zmax+1):
            options.append([x, y, z])
options = np.array(options)
print(f"all possibility: {len(options)}")
print(f"({xmin},{ymin},{zmin}) - ({xmax},{ymax},{zmax})")

options_all = np.copy(options)

# calculate the number of fully-surrounded surfaces.
# # remove cubes which do not have adjacent cubes.
# for adjacent in list_adjacent_surfaces(np.array([0, 0, 0])):
#     options_ = reduce_options(X, options, adjacent)
#     options  = np.copy(options_)

            
# outer surface.
group = []  
try:
    get_group(np.array([xmax, ymax, zmax]))
except RecursionError:
    get_group(np.array([xmax, ymax, zmax]))    
group = np.array(group)

# remove itself.
set_X = np2set(X)
set_options_all = np2set(options_all)
set_group = np2set(group)
options = np.array([x for x in set_options_all - set_X - set_group])

#n_air_pockets = len(options)
#print(f"number of air pockets: {n_air_pockets}")
# print(f"the exterior surface area of the lava droplet: {n_surfaces-6*n_air_pockets}")
# print(f"elapsed time: {time.time()-time_start:.3}[s]")

all possibility: 7220
(1,0,0) - (19,19,18)


In [54]:
n_cubes = len(X)
n_adjacent = calc_adjacent_surfaces(X)
n_surfaces = 6*n_cubes - n_adjacent

# outer surface of the air pockets.
n_air_pockets = len(options)
print(f"number of air pockets: {n_air_pockets}")

n_cubes_inside = len(options)
n_adjacent_inside = calc_adjacent_surfaces(options)
n_surfaces_inside = 6*n_cubes_inside - n_adjacent_inside
print(f"the outer surface area of the air pocket is {n_surfaces_inside}")

print(f"the exterior surface area of the lava droplet: {n_surfaces-n_surfaces_inside}")

number of air pockets: 1836
the outer surface area of the air pocket is 3402
the exterior surface area of the lava droplet: 208


In [56]:
3610-208

3402